# Predicting engine reliability by support vector machines, Wei-Chiang Hong, Ping-Feng Pai (2006)

### Cel pracy:
zbadanie czy maszyny wektorów nośnych nadają się do przewidywania niezawodności silników (enigine reliability)

### Wnioski:
Tak, nadają się.

### Jakieś ważne założenia:
* engine reliability changes with time
* SVMs have been succesfully applied in solving **nonlinear regression and time series problems**


### State of the art:
* ARIMA dla szeregów czasowych w ogóle oraz specyficznie dla problemu ER
* RNNy też niezłe dla ER
* obiecujące eksperymenty na różnych sieciach neuronowych (uwaga: rok 2006)
* SVMS
    * SVMs dla cen na giełdzie, szeregów czasowych w zastosowaniach finansowych, przewidywania jakości powietrza czy prędkości wiatru
    * DSVMs (D stands for dynamic -> [12]) dla niestacjonarych szeregów czasowych
    * C-ascending SVMs dla niestacjonarnych szeregów czasowych w zastosowaniach finansowych

### Modele wybrane przez autorów do porównań:
Autorzy pragną sprawdzić, czy SVMy nadają się do przewidywania niezawodności silników. W tym celu porównają SVMy do:
* Duane
* ARIMA
* GRNN

#### Duane
http://reliawiki.org/index.php/Duane_Model

#### ARIMA
Autoregresive Integrated Moving Average.


#### GRNN
General Regression Neural Network składa się z czterech wartsw:
1. input
2. *pattern neurons* odpowiedzialne za zapamiętywanie relacji między wejściem a pożądanym wyjściem
3. *summation neurons*:
    * $S_s = \sum\limits_i \theta_i$
    * $S_w = \sum\limits_i w_i\theta_i$,
    
    gdzie $w_i$ waga i-tego *pattern* neuronu, $\theta_i$ wyjście i-itego *pattern* neuronu
4. wyjście wyliczane zgodnie ze wzorem: $Q =\frac{S_w}{S_s}$

#### SVM
jak działają SVMy zaproponowane przez autorów?

### Dobór hiperparametrów:
#### Duane

<img src='../img/er_svms/duane.png'>

#### ARIMA
W celu zredukowania ciągą niestacjonarnego do stacjonarnego, wartość parametru $d$ została wybrana na $1$.

Pozostałe dwa parametry: $p = 4$ oraz $q = 3$ zostały wybrane przy użyciu funkcji autokorelacji i funkcji częściowej autokorelacji - vide prezentacja Igora Sieradzkiego. 

Ostatecznie równanie ma postać:

$(1+0.74 B^1 -0.76 B^2 -0.93 B^3 -0.11 B^4) \nabla^1 y_t + 0.0012$ 
$= (1 + 0.95 B^1-0.27b^2-0.72B^3)\epsilon_t$


#### GRNN
Potrzeba jedynie znaleźć parametr wygładzania (*smoothing parameter*) $\sigma$. Sprawdzane jest $n$ wybranych wartości.

<img src='../img/er_svms/grnn_hyps.png' width=500px>

#### SVM
<img src='../img/er_svms/hyperparameters_SVM.png' align='right' style="PADDING-RIGHT: 10px" width=280px>

1. Wybierz przykładową wartość $\epsilon$ oraz przykładową wartość $C$. Wybierz zbiór wartości parametru $\sigma$. Dla wszystkich zestawów naucz SVMa. Zapamiętaj wartość $\sigma'$, dla której model miał najmniejszy błąd NRMSE na zbiorze walidacyjnym.
2. Wybierz przykładową wartość $\epsilon$. Wybierz zbiór wartości dla parametru $C$. $\sigma$ będzie równe $\sigma'$ znalezionemu w poprzednim kroku. Dla wszystkich zestawów naucz SVMa. Zapamiętaj wartość $C'$, dla której model miał najmniejszy błąd NRMSE na zbiorze walidacyjnym.
3. Wybierz zbiór wartości dla hiperparametru $\epsilon$. $\sigma$ będzie równe $\sigma'$ znalezionemu w poprzednim kroku. $C$ będzie równe $C'$ znalezionemu w poprzednim kroku. Dla wszystkich zestawów naucz SVMa. Zapamiętaj wartość $\epsilon'$, dla której model miał najmniejszy błąd NRMSE na zbiorze walidacyjnym.

[obrazek na tablicy w 2D]

Miara: **NRMSE** (*normalised root of mean squared errors*) 
$$NRMSE = \sqrt{\frac{\sum^n_{i=1}(a_i-f_i)^2}{\sum^n_{i=1} a_i^2}}$$.

<img src='../img/er_svms/hiperparametry.png' vspace=30px width=500px>

Zaczęli od $\sigma = 0.5$ oraz $C = 50$. Czemu nie iterowali?


### Dane:

**U.S.S. Grampus Data** a może Halfbeak?

<img src='https://upload.wikimedia.org/wikipedia/commons/b/b1/Grampus_%28SS-523%29.jpg' width=200px align='left' style="PADDING-RIGHT: 10px">

<img src='../img/er_svms/grampus_data.png' width=300px align='right' style="PADDING-LEFT: 10px">

Często wykorzystywany zbiór danych w problemie ER. Grampusy pływały na silnikach Diesla. 

"*Data-set is the data of arrival times to unscheduled maintenance actions  for  the  USS  Grampus  no.  4  main  propulsion  diesel engine*".

Wartością, którą model ma przewidzieć, jest czas następnej awarii silnika.

<img src='../img/er_svms/pub_data.png'>

### Wyniki eksperymentalne:

<img src='../img/er_svms/eksperymenty.png'>


**NRMSE** (normalised root mean-square deviation) $NRMSE = \frac{RMSE}{y_{max}-y_{min}}$.

### Wnioski
SVM użyty przez autorów osiągnął zadowalające wyniki i wydaje się być alternatywą dla dotychczas używanych modeli w problemi ER.

### Dyskusja
* dałoby się znaleźć lepsze hiperparametry dla SVMa
* zbiór wydaje się dosyć skąpy (mniej niż 100 przykładów uczących), co stawia pod pytajnikiem także wyniki eksperymentalne - czy sprawdzenie 14 przykładów daje wynik, który możemy uznać za wiarygodny?
* zresztą czy różnica rzędzy 0.3 NRMSE może zostać użyta za znaczącą?
* autorzy nie piszą o jakiejkolwiek normalizacji danych wejściowych

In [43]:
from sklearn.svm import SVR
import numpy as np

X = np.arange(57).reshape(-1,1)
y = np.loadtxt('data.csv')[:57]

X2 = np.arange(57,71).reshape(-1,1)

clf = SVR(C=60.0, epsilon=0.27, gamma=0.45,kernel='rbf')
clf.fit(X,y)
clf.predict(X2) 

array([ 20.8906465 ,  18.62266375,  17.92552777,  17.84711084,
        17.84366257,  17.84360186,  17.84360143,  17.84360143,
        17.84360143,  17.84360143,  17.84360143,  17.84360143,
        17.84360143,  17.84360143])